In [1]:
%cd /home/q123/Desktop/explo

### local imports 
from src.environment import EnvironmentObjective
from src.vanillagp import step
from src.policy import MyMLP,MyMLP2

### botorch
from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

### general imports
import numpy as np
import gpytorch
import torch
import gym

### Logging 
import logging
logger = logging.getLogger('__main__')
logger.setLevel(logging.WARNING)

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Imports and kernels


In [2]:
### Toy kernel for warningging

class MyKernel(gpytorch.kernels.RBFKernel):
   
    def forward(self,x1,x2,**params):
        
        logger.warning(f'x1 {x1.shape} / x2 {x2.shape}')
        kernel = super().forward(x1,x2,**params)
        logger.warning(f'pair kernel {kernel.shape}')
        return kernel

In [3]:
from gpytorch.kernels import *
from gpytorch.priors.torch_priors import GammaPrior

class GridKernel(Kernel):
    
    def __init__(self,
                 policy,states):
        
        super().__init__()
        
        #tester = PolicyTester(policy,states)
        tester = PolicyTester(states)
        rbf_module = ScaleKernel(RBFKernel(ard_num_dims=states.shape[0]))

        # save params to instance
        self.__dict__.update(locals())
        
        
    def forward(self,x1,x2,**params):
        
        #logger.warning(f'x1 {x1.shape} / x2 {x2.shape}')
        
        #Evaluate current parameters
        
        actions2 = self.tester(x2).squeeze()#temp
        actions1 = self.tester(x1).squeeze()#temp
        #logger.critical(f'actions1 {actions1.shape} actions2 {actions2.shape} ')
        print(f'actions1 {actions1.shape} actions2 {actions2.shape} ')
        
        # Compute pairwise pairwise kernel 
        
        kernel = self.rbf_module(actions1, actions2, **params)
        logger.warning(f'pair kernel {kernel.shape}')
        
        return kernel
        
        
class PolicyTester():
    
    def __init__(self,states):
        
        self.__dict__.update(locals())
    
    def compute_actions(self,states,params_batch):
        
        logger.warning(f'Tester : params_batch.shape{params_batch.shape}')
        mlp = MyMLP([3,1],params_batch)
        return mlp(states)
        
        ## linear policy hotfix
        # print(f'params_batch{params_batch.shape}/ states{states.shape}')
        # return params_batch@states.T
    
    def __call__(self,params_batch):
        
        return self.compute_actions(self.states,
                                    params_batch)


In [4]:
from gpytorch.models import ExactGP
from botorch.models.gpytorch import GPyTorchModel
# We will use the simplest form of GP model, exact inference

class GridGPModel(ExactGP,GPyTorchModel):
    
    _num_outputs = 1
    
    def __init__(self, train_x, train_y, likelihood,
                 mlp,states):
        
        ExactGP.__init__(self, train_x, train_y, likelihood)
        self.covar_module = GridKernel(mlp,states)
        #self.covar_module = MyKernel()
        self.mean_module = gpytorch.means.ConstantMean()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [5]:
# env=gym.make("Pendulum-v1")
# # env.reset()
# # env.step(env.action_space.sample())
# env.__dict__

# box = env._observation_space
# low,high = box.low,box.high 


In [6]:
### initialize policy
mlp = MyMLP2([3,1])

# Initialize environment

objective_env = EnvironmentObjective(
  env=gym.make("Pendulum-v1"),
  policy=mlp,
  manipulate_state=None,
  manipulate_reward=None,
)

### initialize train_x, train_y
train_x = torch.rand(100,mlp.len_params) ## [n_trials,n_params]
train_y = [objective_env.run(p) for p in train_x]
train_y = torch.Tensor(train_y).reshape(-1)  ## [n_trials,1]

# initialize likelihood and model

states = objective_env.get_grid()
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GridGPModel(train_x, train_y, likelihood,
                    mlp,states)

In [7]:
### now we loop :
max_iter = 1

for i in range(max_iter):

  step(model,objective_env)

  if i % 100 == 0:

    best_val = model.train_targets.max()
    curr_val = model.train_targets[-1]
    print(f'curr {curr_val} max {best_val}')

Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Si

MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
actions1 torch.Size([100, 1000]) actions2 torch.Size([100, 1000]) 
MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
actio

Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Size([100, 3])
pair kernel torch.Size([100, 100])
Tester : params_batch.shapetorch.Size([100, 3])
Tester : params_batch.shapetorch.Si

MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
actions1 torch.Size([100, 1000]) actions2 torch.Size([100, 1000]) 
MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
MyMLP received params with shape torch.Size([100, 3])
MyMLP weight size torch.Size([100, 1, 3])
forward: states.shape torch.Size([1000, 3]) params_batch.shape torch.Size([100, 1, 3])
forward: output.shape torch.Size([100, 1, 1000])
actio

RuntimeError: x1_ and x2_ must have the same number of dimensions!

In [ ]:
params_batch = torch.rand(2,9,3)
states = torch.rand(100,3)
(params_batch@states.T).shape

# states = torch.rand(9,3) ## [n_states,state_dim]
# params = torch.rand(3,2,7) ## [in_size,out_size,n_parallel]
# #out = [2,9,7].T ==> [7,9,2]

# states @ params




torch.Size([2, 9, 100])

# Manually fitting GP (maximizing likelihood)

In [ ]:
# training_iter = 100 

# # Find optimal model hyperparameters
# model.train()
# likelihood.train()

# # Use the adam optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.25)  # Includes GaussianLikelihood parameters

# # "Loss" for GPs - the marginal log likelihood
# mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# for i in range(3):
#     # Zero gradients from previous iteration
#     optimizer.zero_grad()
#     # Output from model
#     output = model(train_x)
#     # Calc loss and backprop gradients
#     loss = -mll(output, train_y)
#     logger.warning(f'Loss {loss.shape}')
#     loss.backward()
#     print('Iter %d/%d - Loss: %.3f noise: %.3f' % 
#         (
#         i + 1, training_iter, loss.item(),
#         model.likelihood.noise.item())
#         )
#     optimizer.step()

In [ ]:
# max_iter = 1
# for i in range(max_iter):
      
    
#   ### fit hypers of GP
#   mll = ExactMarginalLogLikelihood(model.likelihood, model)
#   fit_gpytorch_model(mll)
  
#   # logger.setLevel(logging.WARNING)
#   # logger.warning("Optimized hypers")

#   ### optimize acqf
  
#   best_value = model.train_targets.max()
#   len_params = objective_env.policy.len_params
#   EI = ExpectedImprovement(model=model, best_f=best_value)
  
#   new_x, _ = optimize_acqf(
#     acq_function=EI,
#     bounds=torch.tensor([[0.0] * len_params, [1.0] * len_params]),
#     q=1,
#     num_restarts=3,
#     raw_samples=5,
#     options={},
#   )
  
#   logger.setLevel(logging.WARNING)
#   logger.warning("Acquisition function finished")

#   new_y = objective_env(new_x)

#   ### Update training points.
#   train_x = torch.cat([model.train_inputs[0], new_x])
#   train_y = torch.cat([model.train_targets, new_y])
#   model.set_train_data(inputs=train_x, targets=train_y, strict=False)

#   if i % 10 == 0:

#     best_val = model.train_targets.max()
#     curr_val = model.train_targets[-1]
#     print(f'curr {curr_val} max {best_val}')